# Text Classification with TensorFlow Text

In [2]:
try:
  %tensorflow_version 2.x
except Exception:
  pass

# Load the TensorBoard notebook extension
%load_ext tensorboard

TensorFlow 2.x selected.


In [0]:
!pip install tensorflow_text > /dev/null

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import tensorflow_text as tftext
import tensorflow_datasets as tfds
import tensorflow_hub as hub
from tensorflow.keras.layers import Conv1D, Dense, Embedding, Flatten, InputLayer, LSTM, MaxPooling1D, RepeatVector, TimeDistributed
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard
import datetime, os

## Data

### Load the data
Load IMDB reviews dataset using TensorFlow Datasets and splits it into training, validation and test sets.

In [5]:
(train_ds, valid_ds), test_ds = tfds.load(
    name="imdb_reviews",
    split=(tfds.Split.TRAIN.subsplit([8, 2]), tfds.Split.TEST),
    as_supervised=True)

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/0.1.0. Subsequent calls will reuse this data.


### Explore the data
In the IMDb reviews dataset, each example is a sentence representing the movie review and a corresponding label. The sentence is in typical text and need preprocessing before. The label is an integer value representing the polarity of the review: 0 for negative review, and 1 for positive review.

In [6]:
train_text_batch, train_labels_batch = next(iter(train_ds.batch(10)))

pd.DataFrame(data={'text': train_text_batch, 'label': train_labels_batch})

,text,label
0,"b""As a lifelong fan of Dickens, I have invaria...",1
1,"b""Oh yeah! Jenna Jameson did it again! Yeah Ba...",1
2,"b""I saw this film on True Movies (which automa...",1
3,b'This was a wonderfully clever and entertaini...,1
4,b'I have no idea what the other reviewer is ta...,1
5,"b""This was soul-provoking! I am an Iranian, an...",1
6,b'Just because someone is under the age of 10 ...,0
7,"b""I absolutely LOVED this movie when I was a k...",1
8,b'A very close and sharp discription of the bu...,1
9,"b""This is the most depressing film I have ever...",0


### Build vocabulary

#### Download vocab
Load imdb vocabulary

In [7]:
!curl -O http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz && tar -xf aclImdb_v1.tar.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 80.2M  100 80.2M    0     0  10.9M      0  0:00:07  0:00:07 --:--:-- 17.7M


In [8]:
!head aclImdb/imdb.vocab

the
and
a
of
to
is
it
in
i
this


In [9]:
vocab = []
with open('aclImdb/imdb.vocab', 'r') as f:
  cleaned_vocab = map(lambda token: token.replace('\n', ''), f.readlines())
  vocab = list(cleaned_vocab)

vocab[:5]

['the', 'and', 'a', 'of', 'to']

Create a lookup table to encode tokens: `token -> index`

### Build lookup table
Create a token-index mapping to use when encode review text into something that can be processed by a neural network.

In [0]:
def create_vocab_table(vocab, num_oov=1):
  """Create a lookup table for a vocabulary"""
  vocab_values = tf.range(tf.size(vocab, out_type=tf.int64), dtype=tf.int64)
  init = tf.lookup.KeyValueTensorInitializer(keys=vocab, values=vocab_values, key_dtype=tf.string, value_dtype=tf.int64)
  vocab_table = tf.lookup.StaticVocabularyTable(init, num_oov, lookup_key_dtype=tf.string)
  return vocab_table

In [0]:
vocab_table = create_vocab_table(vocab)

### Preprocess the text
Create a preprocessing helper function which will be called on each sample in order to encode the text into a sequence of token indices. 

The output of this transformation is a [RaggedTensor](https://www.tensorflow.org/guide/ragged_tensor).

In [0]:
def basic_preprocess(reviews, labels):
  """Perform basic preprocessing on the reviews text"""
  # Lower case and normalize text
  reviews = tftext.case_fold_utf8(reviews)
  reviews = tftext.normalize_utf8(reviews, "NFD")
  # Tokenize and encode the text
  tokenizer = tftext.UnicodeScriptTokenizer()
  rt = tokenizer.tokenize(reviews)
  # Encode tokens
  features = tf.ragged.map_flat_values(vocab_table.lookup, rt)

  return features, labels

In [0]:
bs = 64

In [14]:
train_ds = train_ds.batch(bs).map(basic_preprocess)
valid_ds = valid_ds.batch(bs).map(basic_preprocess)
test_ds = test_ds.batch(bs).map(basic_preprocess)

Instructions for updating:
`tf.batch_gather` is deprecated, please use `tf.gather` with `batch_dims=-1` instead.


Instructions for updating:
`tf.batch_gather` is deprecated, please use `tf.gather` with `batch_dims=-1` instead.


In [15]:
train_examples_batch, train_labels_batch = next(iter(train_ds))
train_examples_batch.shape, train_labels_batch.shape

(TensorShape([64, None]), TensorShape([64]))

## Model

### RNN-based model
The model to train on this classification task is built using the Sequential API with a staked layers:

1. An `InputLayer` flagged with `ragged` as the model input will be of type `RaggedTensor`.
2. A `ToDense` layer to pad-in inputs
3. An `Embedding` layer to embed the tokens from the review in a lowe space
4. An `LSTM` layer to recursively consume the review sentences

As the task is to classify whether a reivew is positive or negative, a `binary_crossentropy` loss function is used.

#### Build the model

In [0]:
vocab_size, n_units = len(vocab)+1, 50

In [0]:
model = tf.keras.Sequential([
  InputLayer(input_shape=(None,), dtype='int64', ragged=True),
  tftext.keras.layers.ToDense(pad_value=0, mask=True),
  Embedding(vocab_size, n_units),
  LSTM(32),
  Dense(32, activation='relu'),
  Dense(1, activation='sigmoid')
])
model.compile(optimizer="rmsprop", loss="binary_crossentropy", metrics=["accuracy"])

In [24]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
to_dense_1 (ToDense)         (None, None)              0         
_________________________________________________________________
embedding_1 (Embedding)      (None, None, 50)          4476400   
_________________________________________________________________
lstm_1 (LSTM)                (None, 32)                10624     
_________________________________________________________________
dense_2 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 33        
Total params: 4,488,113
Trainable params: 4,488,113
Non-trainable params: 0
_________________________________________________________________


In [0]:
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = TensorBoard(logdir, histogram_freq=1, write_graph=True, write_images=True)

#### Train the model
Train the model for few epochs in mini-batches of 64 samples. Each epoch/iteration we go over all samples (text and label) in the `train_ds` dataset. The validation dataset `valid_ds` is provided so that we can monitor the model's validation loss and accuracy after each epoch. A callback is provided so that information about the training is reported to TensorBoard.

In [26]:
model.fit(train_ds, epochs=10, validation_data=valid_ds, callbacks=[tensorboard_callback])

Epoch 1/10
313/313 [==============================] - 41s 131ms/step - loss: 0.6933 - accuracy: 0.5035 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/10
313/313 [==============================] - 38s 121ms/step - loss: 0.6932 - accuracy: 0.5028 - val_loss: 0.6933 - val_accuracy: 0.4878
Epoch 3/10
313/313 [==============================] - 38s 121ms/step - loss: 0.6924 - accuracy: 0.5035 - val_loss: 0.7118 - val_accuracy: 0.4888
Epoch 4/10
313/313 [==============================] - 38s 122ms/step - loss: 0.6903 - accuracy: 0.5039 - val_loss: 0.6954 - val_accuracy: 0.5156
Epoch 5/10
313/313 [==============================] - 38s 122ms/step - loss: 0.6871 - accuracy: 0.5099 - val_loss: 0.6964 - val_accuracy: 0.5152
Epoch 6/10
313/313 [==============================] - 38s 121ms/step - loss: 0.6887 - accuracy: 0.5107 - val_loss: 0.6947 - val_accuracy: 0.4928
Epoch 7/10
313/313 [==============================] - 38s 121ms/step - loss: 0.6901 - accuracy: 0.5092 - val_loss: 0.6989 

#### Evaluate the model
After training we can check how the model performs on unseen data by checking the loss and accuracy.

In [27]:
results = model.evaluate(test_ds, verbose=2)

for name, value in zip(model.metrics_names, results):
  print(f"{name}: {value}")

391/391 - 18s - loss: 0.7033 - accuracy: 0.5044
loss: 0.703335251039861
accuracy: 0.5044000148773193


### CNN-based model

In [0]:
vocab_size, n_units = len(vocab)+1, 50

In [32]:
model = tf.keras.Sequential([
  InputLayer(input_shape=(None,), dtype='int64', ragged=True),
  tftext.keras.layers.ToDense(pad_value=0, mask=True),
  Embedding(vocab_size, n_units),
  Conv1D(filters=32, kernel_size=8, activation='relu'),
  MaxPooling1D(pool_size=2),
  Flatten(),
  Dense(10, activation='relu'),
  Dense(1, activation='sigmoid')
])
model.compile(optimizer="rmsprop", loss="binary_crossentropy", metrics=["accuracy"])

ValueError: ignored

In [0]:
model.summary()

In [0]:
model.fit(train_ds, epochs=10, validation_data=valid_ds, callbacks=[tensorboard_callback])

#### Evaluate the model
After training we can check how the model performs on unseen data by checking the loss and accuracy.

In [0]:
results = model.evaluate(test_ds, verbose=2)

for name, value in zip(model.metrics_names, results):
  print(f"{name}: {value}")

391/391 - 18s - loss: 0.7033 - accuracy: 0.5044
loss: 0.703335251039861
accuracy: 0.5044000148773193


In [0]:
%tensorboard --logdir logs

## Further reading
For more information about TensorFlow Text check this introduction - [link](https://medium.com/tensorflow/introducing-tf-text-438c8552bd5e)

For general details on RagedTensors check this introduction - [link](https://medium.com/tensorflow/introducing-ragged-tensors-ac301c31fd38)

Example for performing text classification with IMDB dataset - [link](https://www.tensorflow.org/tutorials/keras/text_classification_with_hub)

Example on using pre-trained embedding from TensorFlow Hub - [link](https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1)